# Trabajo Práctico 2: Reseñas de Películas - Random Forest

## Grupo 11 - "Los Outliers"

- Castillo, Carlos
- Destefanis, Juan Pablo
- Gómez, Celeste

# Setup

In [3]:
import pandas as pd

 # Carga de datos

Carga del dataset de entrenamiento anteriormente preprocesado. En este caso, a diferencia de lo que pasó con la red neuronal y XGBoost, obtuvimos mejores resultados con el dataset con preprocesamiento más complejo. Probamos con diferentes combiaciones de las diferentes columnas de este dataset, entre la cuales se encuentran dos variantes del texto preprocesado: `text_cleaned` y `text_cleaned_pos`, que son versiones del texto de la crítica a las que se le aplicó lematización, detección de stop words, regex, unidecode y manejo de las negaciones, sin embargo la diferencia es que la segunda de estas columnas incluye detección de part-of-speech, que nos permite distinguir aún más las palabras debido a su contexto de uso. Además cuenta con columnas que cuentan la cantidad de negaciones en cada crítica, la cantidad de adjetivos negativos y la cantidad de signos de exclamación.

In [4]:
df = pd.read_csv("../datasets/train-random-forest.zip", index_col=0)
df.head()

,review_es,lang,sentimiento,text_cleaned,text_cleaned_pos,num_neg,num_adj_neg,num_exclm
ID,,,,,,,,
0,Uno de los otros críticos ha mencionado que de...,es,positivo,critico mencionar oz episodio estar enganchado...,critico_NOUN mencionar_VERB oz_DET episodio_NO...,7,4,0
1,Una pequeña pequeña producción.La técnica de f...,es,positivo,pequén pequén produccion tecnica filmacion inc...,pequén_ADJ pequén_ADJ produccion_PROPN tecnica...,2,0,2
2,Pensé que esta era una manera maravillosa de p...,es,positivo,pense maravilloso pasar tiempo semana verano c...,pense_VERB maravilloso_ADJ pasar_VERB tiempo_N...,2,0,1
3,"Básicamente, hay una familia donde un niño peq...",es,negativo,basicamente familia nino pequeno jake pensar z...,basicamente_ADV familia_NOUN nino_NOUN pequeno...,0,0,4
4,"El ""amor en el tiempo"" de Petter Mattei es una...",es,positivo,amor tiempo petter_mattei pelicula visualmente...,amor_NOUN tiempo_NOUN pelicula_NOUN visualment...,1,2,0


In [ ]:
from sklearn.model_selection import train_test_split

feature_cols = ["text_cleaned_pos", "num_neg", "num_adj_neg", "num_exclm"]

X = df[feature_cols]
y = df["sentimiento"]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, stratify=y_train, test_size=0.2, random_state=42)

En este caso obtuvimos mejores resultados utilizando ngramas de un rango de 1 a 2.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer

tfidf = TfidfVectorizer(ngram_range=(1, 2), strip_accents="unicode", min_df=20, max_df=0.9)
scaler = StandardScaler()

ct = ColumnTransformer(
    transformers=[
        ("tfidf", tfidf, feature_cols[0]),
        ("scaler", scaler, feature_cols[1:]),
    ],
    remainder="drop",
)

le = LabelEncoder()

In [ ]:
X_train_trans = ct.fit_transform(X_train)
X_val_trans = ct.transform(X_val)
X_test_trans = ct.transform(X_test)

# Entrenamiento del modelo

Los parámetros fueron en primera instancia buscados con RandomizedSearchCV, obteniendo un modelo con mucho overfitting. Con lo cual, se optó por modificar algunos de los parámetros manualmente para reducir ese overfit. Finalmente la mejor combinación a la que llegamos fue la siguiente:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

rf = RandomForestClassifier(
    n_estimators=4000,
    criterion="gini",
    max_depth=40,
    min_samples_split=20,
    min_samples_leaf=16,
    oob_score=True,
    n_jobs=-1
)

rf.fit(X_train_trans, y_train)

y_pred_train = rf.predict(X_train_trans)
y_pred_val = rf.predict(X_val_trans)

X_test_trans = ct.transform(X_test)
y_pred_test = rf.predict(X_test_trans)

f1_score_train = f1_score(y_train, y_pred_train, average="micro")
f1_score_val = f1_score(y_val, y_pred_val, average="micro")
f1_score_test = f1_score(y_test, y_pred_test, average="micro")

Quizá el hyperparámetro que más llama la atención es la gran cantidad de estimadores que utiliza el random forest. Terminamos llegando a valores tan elevados de este parámetro ya que al ir ajustando los parámetros para evitar el overfitting, también iba decreciendo la capacidad del modelo para obtener información de los datos, por lo que todos los parámetros de regularización terminan requiriendo que se incremente la cantidad de árboles en el forest para poder lograr un balance.

In [ ]:
print(f"train      - {f1_score_train:.4}")
print(f"validation - {f1_score_val:.4} - ({(100 - f1_score_val / f1_score_train * 100):.4}% de diferencia con train)")
print(f"test       - {f1_score_test:.4} - ({(100 - f1_score_test / f1_score_train * 100):.4}% de diferencia con train)")

train      - 0.8622
validation - 0.8339 - (3.28% de diferencia con train)
test       - 0.8311 - (3.602% de diferencia con train)


# Predicciones sobre el dataset de testing de la competencia

Finalmente hacemos las predicciones para la competencia de Kaggle.

In [ ]:
import copy

ct_final = copy.deepcopy(ct)

X_final_trans = ct_final.fit_transform(df[feature_cols])
y_final = df["sentimiento"]

rf.fit(X_final_trans, y_final)

RandomForestClassifier(criterion='log_loss', max_depth=35, min_samples_leaf=16,
                       min_samples_split=12, n_estimators=2000, n_jobs=-1,
                       oob_score=True)

In [ ]:
df_kaggle = pd.read_csv("../datasets/random-forest.zip", index_col=0)
df_kaggle["sentimiento"] = rf.predict(ct_final.transform(df_kaggle[feature_cols]))
df_kaggle["sentimiento"]

ID
60000    negativo
60001    negativo
60002    negativo
60003    negativo
60004    positivo
           ...   
68594    positivo
68595    negativo
68596    positivo
68597    negativo
68598    negativo
Name: sentimiento, Length: 8599, dtype: object

In [ ]:
df_kaggle["sentimiento"].to_csv("output.csv")